In [3]:
import sys  
import pandas as pd
from datetime import datetime

In [17]:
matrix_ideam_path = r'../data/variables/data-ideam-month-fill.zip'
matrix_fcl_path = r'../data/variables/FCL316Basins_month_fill.zip'
matrix_output = r'../data/variables/data-month-fill.csv'

In [5]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

In [6]:
df_ideam = pd.read_csv(matrix_ideam_path, parse_dates = ['fecha'], date_parser = dateparse)

In [7]:
df_fcl = pd.read_csv(matrix_fcl_path)

In [8]:
print(df_ideam.shape)
print(df_fcl.shape)

(57600, 13)
(75840, 18)


In [9]:
df = df_ideam.merge(df_fcl, left_on=['cuenca','anio','mes'], right_on=['basin_id','year','month'], how = 'inner' )

In [10]:
print(df.shape)

(57600, 31)


In [11]:
316*12*20

75840

In [12]:
df.drop(columns = [#'fecha', 'cuenca', 
            #'PTPM_CON_max', 'PTPM_CON_median', 'PTPM_CON_min','Q_MEDIA_D_median', 'Q_MN_D_min', 'Q_MX_D_max', 'TMN_CON_min', 'TMX_CON_max', 
           #'elevacion', 'anio', 'mes', 
            'basin_id', 'cod_ah', 'nom_ah', 'cod_szh', 'nom_szh', 'cod_zh', 'nom_zh', 'ruleid',
            'shape_area', 
            #'shape_leng','basin_area', 
            'year', 'month', 'type'
           #'loss_cover_histo', 'loss_cover_fcl', 'loss_cover_fcl_perc','loss_cover_perc_acum'
        ], inplace = True)

In [13]:
df.rename(columns = {'shape_leng': 'basin_perimeter'}, inplace = True)

In [72]:
df[df.cuenca == 4].describe()

,fecha,cuenca,PTPM_CON_max,PTPM_CON_median,PTPM_CON_min,Q_MEDIA_D_median,Q_MN_D_min,Q_MX_D_max,TMN_CON_min,TMX_CON_max,elevacion,anio,mes,basin_perimeter,basin_area,loss_cover_histo,loss_cover_fcl,loss_cover_fcl_perc,loss_cover_perc_acum
42360,2010-01-01,4,16.0,0.0,0.0,0.641,0.616,0.699,10.3,28.3,2467.0,2010,1,1.709663,93254.343,6554.583333,499.971667,0.535833,3.668167


In [89]:
df.to_csv(matrix_output, index = False)

In [91]:
#'elevacion', 'basin_perimeter','basin_area','loss_cover_histo','loss_cover_fcl','loss_cover_fcl_perc','loss_cover_perc_acum'
dfcol = df.groupby(['fecha','anio', 'mes']).median()
dfcol.drop(columns=['cuenca', 'elevacion', 'basin_perimeter', 'basin_area'], inplace = True)

In [93]:
dfcol.to_csv(r'../data/variables/data-colombia-month-fill.csv', index = True)

In [43]:
df_estcuencas = pd.read_csv(r'../data/stations/EST316C.csv', sep=';', dtype = {'CUENCA': 'string'})

In [44]:
df_estcuencas.dtypes

COD        int64
CAT       object
ELEV       int64
CUENCA    string
X         object
Y         object
dtype: object

In [61]:
df_QMN = pd.read_csv(r'../data/variables/var-mes-Q_MN_D-fill.zip', parse_dates = ['Fecha'], date_parser = lambda x: datetime.strptime(x, '%Y-%m'))

In [47]:
df_QMN.shape

(98936, 7)

In [62]:
df_QMN = df_QMN.merge(df_estcuencas, left_on='CodigoEstacion', right_on='COD', how = 'inner' )

In [63]:
df_QMN.shape

(98150, 13)

In [82]:
#df_QMN.loc[df_QMN.CUENCA == '4'].groupby('CodigoEstacion').median.agg(['min', 'max', 'sum'])

cuenca = 4
anio = 2010
mes = 1
display(df[(df.cuenca == cuenca) & (df.anio == anio) & (df.mes == mes)])

display(df_QMN.loc[(df_QMN.CUENCA == str(cuenca)) & (df_QMN.Fecha.dt.year == anio) & (df_QMN.Fecha.dt.month == mes)])
print(df_QMN.loc[(df_QMN.CUENCA == str(cuenca)) & (df_QMN.Fecha.dt.year == anio) & (df_QMN.Fecha.dt.month == mes), 'min'].median())

,fecha,cuenca,PTPM_CON_max,PTPM_CON_median,PTPM_CON_min,Q_MEDIA_D_median,Q_MN_D_min,Q_MX_D_max,TMN_CON_min,TMX_CON_max,elevacion,anio,mes,basin_perimeter,basin_area,loss_cover_histo,loss_cover_fcl,loss_cover_fcl_perc,loss_cover_perc_acum
42360,2010-01-01,4,16.0,0.0,0.0,0.641,0.616,0.699,10.3,28.3,2467.0,2010,1,1.709663,93254.343,6554.583333,499.971667,0.535833,3.668167


,CodigoEstacion,Etiqueta,Fecha,min,mean,median,max,COD,CAT,ELEV,CUENCA,X,Y
49877,26027080,Q_MN_D,2010-01-01,0.434,0.474210,0.467,0.525,26027080,LM,2551,4,"-76,40377778","2,51247222"
50936,26027200,Q_MN_D,2010-01-01,2.230,2.446774,2.440,2.720,26027200,LM,1744,4,"-76,55844444","2,53675"
51642,26027240,Q_MN_D,2010-01-01,0.798,0.798000,0.798,0.798,26027240,LG,2976,4,"-76,33677778","2,47075"


0.7979999999999999


In [83]:
cuencas = [103,140,146,15,154,154,155,166,167,17,179,197,24,243,245,246,252,260,262,27,28,295,299,30,303,36,4,42,44,46,51,65,69,72,75,86,87,89,93,93]

In [87]:
df = df[df.cuenca.isin(cuencas)]

In [88]:
df.head()

,fecha,cuenca,PTPM_CON_max,PTPM_CON_median,PTPM_CON_min,Q_MEDIA_D_median,Q_MN_D_min,Q_MX_D_max,TMN_CON_min,TMX_CON_max,elevacion,anio,mes,basin_perimeter,basin_area,loss_cover_histo,loss_cover_fcl,loss_cover_fcl_perc,loss_cover_perc_acum
1200,2000-01-01,103,40.0,2.0,0.0,14.48,8.76,35.48,NaN,NaN,1834.0,2000,1,1.876566,60038.081,NaN,NaN,NaN,NaN
1201,2000-02-01,103,50.0,4.0,0.0,10.94,7.08,40.52,NaN,NaN,1834.0,2000,2,1.876566,60038.081,NaN,NaN,NaN,NaN
1202,2000-03-01,103,38.0,3.0,0.0,10.68,6.60,20.54,NaN,NaN,1834.0,2000,3,1.876566,60038.081,NaN,NaN,NaN,NaN
1203,2000-04-01,103,25.0,2.0,0.0,9.00,6.60,16.56,NaN,NaN,1834.0,2000,4,1.876566,60038.081,NaN,NaN,NaN,NaN
1204,2000-05-01,103,23.0,2.0,0.0,10.30,6.60,25.16,NaN,NaN,1834.0,2000,5,1.876566,60038.081,NaN,NaN,NaN,NaN
